In [1]:
from sage.interfaces.kenzo import *

In [3]:
# #Testing Sphere
# s1= Sphere(2)
# print(s1.orgn)

# #Testing Moore space
# m24 = MooreSpace(2,4)
# print(m24.orgn)

# #Testing EilenbergMacLaneSpace
# e3 = EilenbergMacLaneSpace(ZZ, 3)
# print(e3.orgn)

# #Testing EilenbergMacLaneSpace
# f3 = EilenbergMacLaneSpace(AdditiveAbelianGroup([2]), 3)
# print(f3.orgn)

# #Testing tensor product
# s2 = Sphere(2)
# s3 = Sphere(3)
# p = s2.tensor_product(s3)
# print(p.orgn)

# #Testing loop space
# l2 = s2.loop_space()
# print(l2.orgn)

# #Testing cartesian product
# s2 = Sphere(2)
# s3 = Sphere(3)
# p = s2.cartesian_product(s3)
# print(p.orgn)


# #Testing Suspension
# s = e3.suspension()
# print(s.orgn)

# #Testing wedge
# s2 = Sphere(2)
# s3 = Sphere(3)
# p = s2.wedge(s3)
# print(p.orgn)

# #Testing join
# s2 = Sphere(2)
# s3 = Sphere(3)
# p = s2.join(s3)
# print(p.orgn)


# #Testing smash product
# s2 = Sphere(2)
# s3 = Sphere(3)
# p = s2.smash_product(s3)
# print(p.orgn)

# #Testing classifying space
# c = l2.classifying_space()
# print(c.orgn)

#__tnsr_prdc__ = EclObject("tnsr-prdc")
#__sphere__ = EclObject("sphere")
#__tnsr_prdc__(__sphere__(2),__sphere__(3))

__sphere__(2)
__moore__(2,4)
__k_z__(3)
__k_z2__(3)
__tnsr_prdc__(__sphere__(2),__sphere__(3))
__loop_space__(__sphere__(2),1)
__crts_prdc__(__sphere__(2),__sphere__(3))
__suspension__(__k_z__(3))
__wedge__(__sphere__(2),__sphere__(3))
__join__(__sphere__(2),__sphere__(3))
__smash_product__(__sphere__(2),__sphere__(3))
__classifying_space__(__loop_space__(__sphere__(2),1))


In [2]:
from sage.interfaces.kenzo import *
__echcm__ = EclObject("echcm")
__chcm_mat__ = EclObject("chcm-mat")
__homologie__ = EclObject("homologie")
__convertmatrice__ = EclObject("convertmatrice")
__convertarray__ = EclObject("convertarray")

In [3]:
from multiprocessing import Pool
import multiprocessing

def __chcm_mat2__(KenzoChainComplex, n):
    echcm1 = __echcm__(KenzoChainComplex._kenzo)
    m = __chcm_mat__(echcm1, n)
    return(k2s_matrix(__convertmatrice__(m)))
    

def chm_mat_parallel3(l):
    pool = Pool(2)
    final = pool.starmap(__chcm_mat2__, l)
    return final

In [4]:
import time

def homology_parallel(KenzoChainComplex, n):
    m1, m2 = chm_mat_parallel3([(KenzoChainComplex, n), (KenzoChainComplex, n+1)])

    #Convert again to sparse Kenzo matrix (matrice). Es decir, el inverso de __convertmatrice__.
    
    m1 = __convertarray__(s2k_matrix(m1))
    m2 = __convertarray__(s2k_matrix(m2))
    
    homology = __homologie__(m1, m2)
    lhomomology = [i for i in EclListIterator(homology)]
    res = []
    for component in lhomomology:
        pair = [i for i in EclListIterator(component)]
        res.append(pair[0].python())
    return HomologyGroup(len(res), ZZ, res)

## MULTIPROCESSING ANA (COLUMNAS)

In [5]:
import time
def __chcm_mat_3__(KenzoChainComplex, n, thread, threads):
    ecl_eval("""(DEFUN CHCM-MAT-COLUMNS (chcm n first-index last-index)
  (declare
   (type chain-complex chcm)
   (type fixnum n))
  (let* ((all-sorc (basis chcm n))
        (sorc (subseq all-sorc first-index last-index))
        (trgt (basis chcm (1- n)))
        (cmpr (cmpr chcm)))
    (declare
     (list sorc trgt)
     (type cmprf cmpr))
    (let ((sorcl (length sorc))
          (mat (creer-matrice (length trgt) (length sorc)))
          (test #'(lambda (gnrt1 gnrt2)
                    (eq (funcall cmpr gnrt1 gnrt2) :equal))))
      (declare
       (type fixnum sorcl)
       (type matrice mat)
       (function test))
      (do ((i 1 (1+ i))
           (mark sorc (cdr mark)))
          ((endp mark))
        (declare
         (type fixnum i)
         (list mark))
          (maj-colonne mat i
             (mapcar #'(lambda (term)
                          (declare (type term term))
                          (list
                             (1+ (position (gnrt term) trgt :test test))
                             (cffc term)))
               (cmbn-list (? chcm n (car mark))))))
       mat)))""")
    __chcm_mat_columns__ = EclObject("CHCM-MAT-COLUMNS")
    echcm1 = __echcm__(KenzoChainComplex._kenzo)
    #Number of cols = (setf ncol (length (basis echcm n)))
    __length__ = EclObject("length")
    __basis__ = EclObject("basis_aux1")
    ncols = __length__(__basis__(echcm1,n)).python()
        
    interval = max(int(ncols) // threads, 1)
    if thread < ncols:
        if thread + 1 == threads:
            m = __chcm_mat_columns__(echcm1, n, thread * interval, ncols)
        else:
            m = __chcm_mat_columns__(echcm1, n, thread * interval, (thread + 1)*interval)        
        #print(k2s_matrix(__convertmatrice__(m)))
        return(k2s_matrix(__convertmatrice__(m)))
    else:
        return None
    

def chm_mat_parallel_columns(l):
    pool = Pool()
    final = pool.starmap(__chcm_mat_3__, l)
    return final

In [6]:
def homology_parallel_columns(KenzoChainComplex, n, threads = multiprocessing.cpu_count()):
    
    #TODO: One workers list with all tasks (this would permit more parallelism)
    workers = zip([KenzoChainComplex]*threads,[n]*threads,range(threads), [threads]*threads)    
    cols1 = chm_mat_parallel_columns(workers)
    
    workers = zip([KenzoChainComplex]*threads,[n+1]*threads,range(threads), [threads]*threads)    
    cols2 = chm_mat_parallel_columns(workers)
    
    cols1 = [item for item in cols1 if item is not None] #Delete None elements    
    m1 = cols1[0]
    for c in cols1[1:]: #Skip the first (it is already m1)    
        #if c is not None:
        m1 = m1.augment(c)         
  
    cols2 = [item for item in cols2 if item is not None] #Delete None elements
    m2 = cols2[0]
    for c in cols2[1:]: #Skip the first (it is already m2)    
        #if c is not None:
        m2 = m2.augment(c)         
            
    #Convert again to sparse Kenzo matrix (matrice). Es decir, el inverso de __convertmatrice__.
    
    m1 = __convertarray__(s2k_matrix(m1))
    m2 = __convertarray__(s2k_matrix(m2))
    
    homology = __homologie__(m1, m2)
    lhomomology = [i for i in EclListIterator(homology)]
    res = []
    for component in lhomomology:
        pair = [i for i in EclListIterator(component)]
        res.append(pair[0].python())
    return HomologyGroup(len(res), ZZ, res)

In [7]:
KenzoChainComplex.homology_parallel=homology_parallel
KenzoChainComplex.homology_parallel_columns=homology_parallel_columns

In [8]:
e3 = EilenbergMacLaneSpace(ZZ, 3)

s1 = Sphere(3)
e = s1.cartesian_product(e3.loop_space())

print("Available CPUs:", multiprocessing.cpu_count())

inicial = time.time()
H = e.homology_parallel_columns(10)
print(H)
print("Time parallelism (columns version)", time.time() - inicial)

inicial = time.time()
H = e.homology_parallel(10)
print(H)
print("Time parallelism (basic version)", time.time() - inicial)

inicial = time.time()
H = e.homology(10)
print(H)
print("Time NO parallelism", time.time() - inicial)

Available CPUs: 8
Z
Time parallelism (columns version) 92.53180241584778
Z
Time parallelism (basic version) 102.87789940834045
Z
Time NO parallelism 114.16639590263367


In [9]:

kz3 = EilenbergMacLaneSpace(ZZ, 3)
kzp_6 = EilenbergMacLaneSpace(AdditiveAbelianGroup([5]), 7)

e = kz3.cartesian_product(kzp_6)

print("Available CPUs:", multiprocessing.cpu_count())

inicial = time.time()
H = e.homology_parallel_columns(10)
print(H)
print("Time parallelism (columns version)", time.time() - inicial)

inicial = time.time()
H = e.homology_parallel(10)
print(H)
print("Time parallelism (basic version)", time.time() - inicial)

inicial = time.time()
H = e.homology(10)
print(H)
print("Time NO parallelism", time.time() - inicial)

Available CPUs: 8
C3 x C5
Time parallelism (columns version) 2.0263073444366455
C3 x C5
Time parallelism (basic version) 3.0132884979248047
C3 x C5
Time NO parallelism 2.930284023284912


In [10]:

kz3 = EilenbergMacLaneSpace(ZZ, 3)
kzp_6 = EilenbergMacLaneSpace(AdditiveAbelianGroup([5]), 7)

e = kz3.cartesian_product(kzp_6)

print("Available CPUs:", multiprocessing.cpu_count())

inicial = time.time()
H = e.homology_parallel_columns(12)
print(H)
print("Time parallelism (columns version)", time.time() - inicial)

inicial = time.time()
H = e.homology_parallel(12)
print(H)
print("Time parallelism (basic version)", time.time() - inicial)

inicial = time.time()
H = e.homology(12)
print(H)
print("Time NO parallelism", time.time() - inicial)

Available CPUs: 8
C2
Time parallelism (columns version) 109.7536051273346
C2
Time parallelism (basic version) 325.527046918869
C2
Time NO parallelism 330.58279490470886


## EXPERIMENTS

In [11]:
f = open("benchmarks_kz3_kzp6.txt", "w")
f.write("Dimension;Time_parallelism_columns;Time_parallelism_basic;Time_no_parallelism\n")
for i in range(7,14):
    ecl_eval("(cat-init)") #To avoid caching computations in Kenzo
    
    kz3 = EilenbergMacLaneSpace(ZZ, 3)
    kzp_6 = EilenbergMacLaneSpace(AdditiveAbelianGroup([5]), 7)
    e = kz3.cartesian_product(kzp_6)

    inicial_paralel_columns = time.time()
    H = e.homology_parallel_columns(i)
    final_paralel_columns = time.time() - inicial_paralel_columns

    inicial_parallel = time.time()
    H = e.homology_parallel(i)
    final_parallel = time.time() - inicial_parallel
    

    inicial_no_parallelism = time.time()
    H = e.homology(i)
    final_no_parallelism = time.time() - inicial_no_parallelism
    
    f.write(str(i) + ";" + str(final_paralel_columns) + ";" + str(final_parallel) + ";" + str(final_no_parallelism) + "\n")
    f.flush()
    
f.close()

In [ ]:
f = open("benchmarks_kz3_loop.txt", "w")
f.write("Dimension;Time_parallelism_columns;Time_parallelism_basic;Time_no_parallelism\n")
for i in range(3,11):
    ecl_eval("(cat-init)") #To avoid caching computations in Kenzo
    
    e3 = EilenbergMacLaneSpace(ZZ, 3)
    s1 = Sphere(3)
    e = s1.cartesian_product(e3.loop_space())

    inicial_paralel_columns = time.time()
    H = e.homology_parallel_columns(i)
    final_paralel_columns = time.time() - inicial_paralel_columns

    inicial_parallel = time.time()
    H = e.homology_parallel(i)
    final_parallel = time.time() - inicial_parallel
    

    inicial_no_parallelism = time.time()
    H = e.homology(i)
    final_no_parallelism = time.time() - inicial_no_parallelism
    
    f.write(str(i) + ";" + str(final_paralel_columns) + ";" + str(final_parallel) + ";" + str(final_no_parallelism) + "\n")
    f.flush()
    
f.close()

In [ ]:
f = open("benchmarks_join_wedge.txt", "w")
f.write("Dimension;Time_parallelism_columns;Time_parallelism_basic;Time_no_parallelism\n")
for i in range(3,8):
    ecl_eval("(cat-init)") #To avoid caching computations in Kenzo
        
    s2 = Sphere(2)
    s3 = Sphere(3)
    s4 = Sphere(4)
    s5 = Sphere(5)
    l2 = s2.wedge(s3)
    e = l2.join(l2.loop_space())

    inicial_paralel_columns = time.time()
    H = e.homology_parallel_columns(i)
    final_paralel_columns = time.time() - inicial_paralel_columns

    inicial_parallel = time.time()
    H = e.homology_parallel(i)
    final_parallel = time.time() - inicial_parallel
    

    inicial_no_parallelism = time.time()
    H = e.homology(i)
    final_no_parallelism = time.time() - inicial_no_parallelism
    
    f.write(str(i) + ";" + str(final_paralel_columns) + ";" + str(final_parallel) + ";" + str(final_no_parallelism) + "\n")
    f.flush()
    
f.close()

## PROFILING

In [8]:
import time

def homology_times(self, n):
        r"""
        Return the ``n``'th homology group of the chain complex associated to this
        kenzo object.

        INPUT:

        - ``n`` -- the dimension in which compute the homology

        OUTPUT:

        - An homology group.

        EXAMPLES::

            sage: from sage.interfaces.kenzo import Sphere   # optional - kenzo
            sage: s2 = Sphere(2)                             # optional - kenzo
            sage: s2                                         # optional - kenzo
            [K1 Simplicial-Set]
            sage: s2.homology(2)                             # optional - kenzo
            Z
        """
        inicial = time.time()
        echcm1 = __echcm__(self._kenzo)
        inicial_chcm_mat = time.time()
        m1 = __chcm_mat__(echcm1, n)
        final_m1 = time.time() - inicial_chcm_mat
        inicial_m2 = time.time()
        m2 = __chcm_mat__(echcm1, n + 1)
        final_m2 = time.time() - inicial_m2
        final_chcm_mat= time.time() - inicial_chcm_mat
        print("FINAL inicial_chcm_mat", final_chcm_mat)
        print("m1", final_m1)
        print("m2", final_m2)
        homology = __homologie__(m1, m2)
        lhomomology = [i for i in EclListIterator(homology)]
        res = []
        for component in lhomomology:
            pair = [i for i in EclListIterator(component)]
            res.append(pair[0].python())
        final_total = time.time() - inicial        
        print("FINAL Total", final_total)
        print("Porcentaje", final_chcm_mat/final_total)
        return HomologyGroup(len(res), ZZ, res)

In [ ]:
%prun  _=e3.homology_times(13)

In [9]:
KenzoChainComplex.homology_times=homology_times
e3 = EilenbergMacLaneSpace(ZZ, 3)
e3.homology_times(12)


FINAL inicial_chcm_mat 15.598355531692505
m1 3.3446266651153564
m2 12.253724813461304
FINAL Total 15.612226009368896
Porcentaje 0.9991115631000943


C2

In [42]:
e3 = EilenbergMacLaneSpace(ZZ, 3)
e3.homology_times(10)